In [5]:
%%bash
rm libsndfile-1.0.28.tar.gz
rm -fr libsndfile-1.0.28
wget http://www.mega-nerd.com/libsndfile/files/libsndfile-1.0.28.tar.gz
tar -xzvf ./libsndfile-1.0.28.tar.gz
cd ./libsndfile-1.0.28
./configure --prefix=/usr    \
            --disable-static \
            --docdir=/usr/share/doc/libsndfile-1.0.28 &&
make
sudo make install
rm libsndfile-1.0.28.tar.gz
rm -fr libsndfile-1.0.28
cd ..

libsndfile-1.0.28/
libsndfile-1.0.28/aclocal.m4
libsndfile-1.0.28/CMakeLists.txt
libsndfile-1.0.28/CMake/
libsndfile-1.0.28/CMake/build.cmake
libsndfile-1.0.28/CMake/compiler_is_gcc.c
libsndfile-1.0.28/CMake/file.cmake
libsndfile-1.0.28/CMake/libsndfile.cmake
libsndfile-1.0.28/CMake/check.cmake
libsndfile-1.0.28/CMake/autogen.cmake
libsndfile-1.0.28/CMake/have_decl_s_irgrp.c
libsndfile-1.0.28/CMake/external_libs.cmake
libsndfile-1.0.28/M4/
libsndfile-1.0.28/M4/lt~obsolete.m4
libsndfile-1.0.28/M4/clip_mode.m4
libsndfile-1.0.28/M4/libtool.m4
libsndfile-1.0.28/M4/llrint.m4
libsndfile-1.0.28/M4/octave.m4
libsndfile-1.0.28/M4/extra_pkg.m4
libsndfile-1.0.28/M4/ltsugar.m4
libsndfile-1.0.28/M4/add_cxxflags.m4
libsndfile-1.0.28/M4/ltoptions.m4
libsndfile-1.0.28/M4/clang.m4
libsndfile-1.0.28/M4/add_cflags.m4
libsndfile-1.0.28/M4/lrint.m4
libsndfile-1.0.28/M4/stack_protect.m4
libsndfile-1.0.28/M4/gcc_version.m4
libsndfile-1.0.28/M4/Makefile.am
libsndfile-1.0.28/M4/endian.m4
libsndfile-1.0.28/M4/l

rm: cannot remove ‘libsndfile-1.0.28.tar.gz’: No such file or directory
--2020-06-25 02:38:45--  http://www.mega-nerd.com/libsndfile/files/libsndfile-1.0.28.tar.gz
Resolving www.mega-nerd.com (www.mega-nerd.com)... 97.107.128.207
Connecting to www.mega-nerd.com (www.mega-nerd.com)|97.107.128.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1202833 (1.1M) [application/x-gzip]
Saving to: ‘libsndfile-1.0.28.tar.gz’

     0K .......... .......... .......... .......... ..........  4%  365K 3s
    50K .......... .......... .......... .......... ..........  8%  730K 2s
   100K .......... .......... .......... .......... .......... 12% 64.0M 1s
   150K .......... .......... .......... .......... .......... 17%  268M 1s
   200K .......... .......... .......... .......... .......... 21%  735K 1s
   250K .......... .......... .......... .......... .......... 25%  118M 1s
   300K .......... .......... .......... .......... .......... 29%  110M 1s
   350K .......... ....

In [6]:
!pip install librosa --user
#!pip install sndfile

You are using pip version 10.0.1, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import os
import subprocess
import sagemaker
import torch
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()
role = get_execution_role()

bucket = 'uwo-bkt-xj' # custom bucket name.
prefix = 'audio-extraction'
s3_data_path = "s3://{}/{}".format(bucket, prefix)
print(s3_data_path)

s3://uwo-bkt-xj/audio-extraction


## 准备数据

In [3]:
import librosa
import json
sample_rate=16000
def generate_json(in_dir,out_dir,out_filename,ttype):
    file_infos = []
    in_dir = os.path.abspath(in_dir)
    wav_list = os.listdir(in_dir)
    for wav_file in wav_list:
        if not wav_file.endswith('.wav'):
            continue
        wav_path = os.path.join(in_dir, wav_file)
        samples, _ = librosa.load(wav_path, sr=sample_rate)
        fake_wav_path = os.path.join('/opt/ml/input/data', ttype, out_filename, wav_file)
        file_infos.append((fake_wav_path, len(samples)))
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    with open(os.path.join(out_dir, out_filename + '.json'), 'w') as f:
        json.dump(file_infos, f, indent=4)
        
generate_json('train_sample_v6/tr/mix_both','train_sample_v6/tr','mix_both','train')
generate_json('train_sample_v6/tr/s1','train_sample_v6/tr','s1','train')
generate_json('train_sample_v6/tr/s2','train_sample_v6/tr','s2','train')
generate_json('train_sample_v6/cv/mix_both','train_sample_v6/cv','mix_both','test')
generate_json('train_sample_v6/cv/s1','train_sample_v6/cv','s1','test')
generate_json('train_sample_v6/cv/s2','train_sample_v6/cv','s2','test')

!aws s3 sync ./train_sample_v6/tr/ s3://uwo-bkt-xj/audio-extraction/train
!aws s3 sync ./train_sample_v6/cv/ s3://uwo-bkt-xj/audio-extraction/test

upload: train_sample_v6/tr/mix_both.json to s3://uwo-bkt-xj/audio-extraction/train/mix_both.json
upload: train_sample_v6/tr/mix_both/1-40-t-n-1_1-41-nav-n-1_0dB.wav to s3://uwo-bkt-xj/audio-extraction/train/mix_both/1-40-t-n-1_1-41-nav-n-1_0dB.wav
upload: train_sample_v6/tr/mix_both/1-35-t-s-n-4_1-46-nav-n-2_0dB.wav to s3://uwo-bkt-xj/audio-extraction/train/mix_both/1-35-t-s-n-4_1-46-nav-n-2_0dB.wav
upload: train_sample_v6/tr/s1.json to s3://uwo-bkt-xj/audio-extraction/train/s1.json
upload: train_sample_v6/tr/mix_both/1-50-t-s-n-2_1-36-nav-n-2_0dB.wav to s3://uwo-bkt-xj/audio-extraction/train/mix_both/1-50-t-s-n-2_1-36-nav-n-2_0dB.wav
upload: train_sample_v6/tr/mix_both/1-40-t-n-1_1-37-nav-n-1_0dB.wav to s3://uwo-bkt-xj/audio-extraction/train/mix_both/1-40-t-n-1_1-37-nav-n-1_0dB.wav
upload: train_sample_v6/tr/mix_both/1-46-t-s-n-1_1-46-nav-n-1_0dB.wav to s3://uwo-bkt-xj/audio-extraction/train/mix_both/1-46-t-s-n-1_1-46-nav-n-1_0dB.wav
upload: train_sample_v6/tr/mix_both/1-23-t-m-n-1_1-

## 开始训练和部署流程
### 设置训练和验证数据路径

In [4]:
# # #anbei
# train_inputs="s3://sagemaker-us-east-2-310850127430/shouqi/train/"
# test_inputs="s3://sagemaker-us-east-2-310850127430/shouqi/test/"
#aoyu
#train_inputs="s3://sagemaker-us-east-1-002224604296/separate/train/"
#test_inputs="s3://sagemaker-us-east-1-002224604296/separate/test/"
#xj
train_inputs="s3://uwo-bkt-xj/audio-extraction/train/"
test_inputs="s3://uwo-bkt-xj/audio-extraction/test/"

### 利用sagemaker pytorch进行训练

In [5]:
from sagemaker.pytorch import PyTorch

dprnn_estimator = PyTorch(entry_point='dprnn-train-deploy.py',
                            source_dir='code',
                            train_instance_type='ml.p3.2xlarge',
                            train_instance_count=1,
                            role=role,
                            framework_version='1.5.1',
                            hyperparameters = {'epochs': 1})
dprnn_estimator.fit({'train': train_inputs,
                       'test': test_inputs})

2020-06-25 02:40:38 Starting - Starting the training job...
2020-06-25 02:40:40 Starting - Launching requested ML instances......
2020-06-25 02:41:51 Starting - Preparing the instances for training......
2020-06-25 02:42:56 Downloading - Downloading input data...
2020-06-25 02:43:21 Training - Downloading the training image........bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-06-25 02:44:47,228 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2020-06-25 02:44:47,250 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-06-25 02:44:47,472 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-06-25 02:44:47,747 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python -m pip install -r requirements.txt

2020-06-25 02:44:46 Training - Training image download comple

### 将训练好的模型进行部署

In [6]:
# Deploy my estimator to a SageMaker Endpoint and get a Predictor
predictor = dprnn_estimator.deploy(instance_type='ml.g4dn.xlarge',
                                     initial_instance_count=1)

---------------!

### 在部署好的模型上进行推理

In [7]:
from helper.utils import *

# 10 seconds for each 
instance_time = 10
filename = './test_sample/audio_raw_1-20_clip_0.wav'
# filename = './data/tt/mix/1-20-t-1_1-20-t-1_0dB.wav'

x, fs = preprocess_wav(filename)
instance_seg = instance_time * fs

In [9]:
l_x = len(x)
s_x = instance_seg
s1 = np.array([])
s2 = np.array([])
gl_i = 0
for start in range(0, l_x-s_x, s_x):
    flat_estimate = separate_process(x[start:start+s_x], predictor)
    s1 = np.concatenate([s1, flat_estimate[0]])
    s2 = np.concatenate([s2, flat_estimate[1]])
    gl_i = start+s_x
if gl_i < l_x:
    flat_estimate = separate_process(x[gl_i:], predictor)
    s1 = np.concatenate([s1, flat_estimate[0]])
    s2 = np.concatenate([s2, flat_estimate[1]])
    
write(s1, './result/complete_split_s1.wav')
write(s2, './result/complete_split_s2.wav')